# kramer2.0 Experiment 01: Analyses
## Angelique I. Delarazan
### kramer2.0_experiment_01_vB5
### Retrieval Tasks: Character-Cued Recall and Event Distance Task

## Set Up

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from scipy import stats
import itertools
import statsmodels.api as sm 
from statsmodels.formula.api import ols 
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import pingouin as pg
from pingouin import anova as pg_anova
from statannot import add_stat_annotation

/Users/aidelarazan/miniconda3/envs/kramer/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


## Charcter-Cued Recall

In [2]:
recall = pd.read_csv('~/Box Sync/aidelarazan_box/Projects/kramer2.0/data/kramer2.0_vB5/kramer2.0_vB5_sub-all_desc-recall.csv')
recall['condition'] = recall['coherence'] + '_' + recall['lag']
print(recall.columns.tolist())
recall.head()

['subject', 'version', 'character', 'coherence', 'lag', 'plot', 'response', 'rt', 'word_count', 'condition']


,subject,version,character,coherence,lag,plot,response,rt,word_count,condition
0,301,6,Melvin,unrelated,long,sideplot,"{""Q0"":""Melvin needed a recipe from his grandma...",189901.7,71,unrelated_long
1,301,6,Beatrice,coherent,short,sideplot,"{""Q0"":""Fist, she asked Charles to chose betwee...",293047.3,73,coherent_short
2,301,6,Johnny,coherent,long,sideplot,"{""Q0"":""Johnny called Charles to tell him about...",181397.2,39,coherent_long
3,301,6,Sandra,unrelated,short,sideplot,"{""Q0"":""Sandra first asked for Charles opinion ...",181062.9,89,unrelated_short
4,301,6,Charles,mainplot,mainplot,mainplot,"{""Q0"":""Charles was wearing a beige sport coat ...",442870.5,226,mainplot_mainplot


In [3]:
aov = pg.rm_anova(
    data=recall[recall['coherence']!='mainplot'],
    dv='word_count',
    within=['coherence', 'lag'],
    subject='subject',
    detailed=True
)

pg.print_table(aov)

post_hoc = pg.pairwise_tests(
    data=recall[recall['coherence']!='mainplot'],
    dv='word_count',
    within='condition',
    subject='subject',
    padjust='bonf'
)

pg.print_table(post_hoc)


ANOVA SUMMARY

Source                  SS    ddof1    ddof2         MS       F    p-unc    p-GG-corr    ng2    eps
---------------  ---------  -------  -------  ---------  ------  -------  -----------  -----  -----
coherence        33586.673        1       38  33586.673  37.065    0.000        0.000  0.116  1.000
lag                  0.160        1       38      0.160   0.000    0.989        0.989  0.000  1.000
coherence * lag      0.776        1       38      0.776   0.001    0.971        0.971  0.000  1.000


POST HOC TESTS

Contrast    A               B                Paired    Parametric         T     dof  alternative      p-unc    p-corr  p-adjust        BF10    hedges
----------  --------------  ---------------  --------  ------------  ------  ------  -------------  -------  --------  ----------  --------  --------
condition   coherent_long   coherent_short   True      True           0.013  38.000  two-sided        0.990     1.000  bonf           0.173     0.002
condition   cohe

In [4]:
ttest = pg.ttest(
    x=recall[recall['coherence']=='coherent'].groupby(['subject', 'coherence', 'lag', 'condition']).sum().reset_index()['word_count'],
    y=recall[recall['coherence']=='unrelated'].groupby(['subject', 'coherence', 'lag', 'condition']).sum().reset_index()['word_count'],
    paired=False
)

pg.print_table(ttest)

print()

    T    dof  alternative      p-val  CI95%            cohen-d      BF10    power
-----  -----  -------------  -------  -------------  ---------  --------  -------
4.488    154  two-sided        0.000  [16.43 42.26]      0.719  1249.942    0.994




In [5]:
ttest = pg.ttest(
    x=recall[recall['lag']=='short'].groupby(['subject', 'coherence', 'lag', 'condition']).sum().reset_index()['word_count'],
    y=recall[recall['lag']=='long'].groupby(['subject', 'coherence', 'lag', 'condition']).sum().reset_index()['word_count'],
    paired=False
)

pg.print_table(ttest)

print()

    T    dof  alternative      p-val  CI95%              cohen-d    BF10    power
-----  -----  -------------  -------  ---------------  ---------  ------  -------
0.009    154  two-sided        0.993  [-13.67  13.8 ]      0.001   0.173    0.050




## Event Distance Task

In [6]:
distance = pd.read_csv('~/Box Sync/aidelarazan_box/Projects/kramer2.0/data/kramer2.0_vB5/kramer2.0_vB5_sub-all_desc-distance.csv')
distance['condition'] = distance['coherence'] + '_' + distance['lag']
print(distance.columns.tolist())
distance.head()

['subject', 'version', 'character_1', 'act_1', 'event_1', 'image_1', 'position_1', 'character_2', 'act_2', 'event_2', 'image_2', 'position_2', 'coherence', 'lag', 'plot', 'response', 'correct_distance', 'rt', 'stimulus', 'condition', 'trial_index', 'error', 'abs_error']


,subject,version,character_1,act_1,event_1,image_1,position_1,character_2,act_2,event_2,...,lag,plot,response,correct_distance,rt,stimulus,condition,trial_index,error,abs_error
0,301,6,Charles,1,E,3,67,Charles,2,A,...,mainplot,NaN,118,8,7400.5,"[""<img src=\""jspsych/stimuli/charles1E3.png\"" ...",mainplot_mainplot,25,110,110
1,301,6,Charles,1,B,2,18,Charles,1,B,...,mainplot,NaN,114,2,5674.9,"[""<img src=\""jspsych/stimuli/charles1B2.png\"" ...",mainplot_mainplot,26,112,112
2,301,6,Sandra,1,A,2,58,Sandra,2,B,...,short,NaN,101,42,7674.4,"[""<img src=\""jspsych/stimuli/sandra1A2.png\"" w...",unrelated_short,27,59,59
3,301,6,Johnny,1,A,7,31,Johnny,2,A,...,long,NaN,90,103,9193.8,"[""<img src=\""jspsych/stimuli/johnny1A7.png\"" w...",coherent_long,28,-13,13
4,301,6,Sandra,1,A,8,64,Sandra,2,B,...,short,NaN,87,34,9309.0,"[""<img src=\""jspsych/stimuli/sandra1A8.png\"" w...",unrelated_short,29,53,53


In [7]:
distance = distance[distance['coherence']!='mainplot'].groupby(['subject', 'coherence', 'lag', 'condition']).mean().reset_index()

In [8]:
aov = pg.rm_anova(
    data=distance[distance['coherence']!='mainplot'],
    dv='error',
    within=['coherence', 'lag'],
    subject='subject',
    detailed=True
)

pg.print_table(aov)

post_hoc = pg.pairwise_tests(
    data=distance[distance['coherence']!='mainplot'],
    dv='error',
    within='condition',
    subject='subject',
    padjust='bonf'
)

pg.print_table(post_hoc)


ANOVA SUMMARY

Source                  SS    ddof1    ddof2         MS        F    p-unc    p-GG-corr    ng2    eps
---------------  ---------  -------  -------  ---------  -------  -------  -----------  -----  -----
coherence          161.295        1       38    161.295    1.225    0.275        0.275  0.006  1.000
lag              61178.977        1       38  61178.977  694.677    0.000        0.000  0.689  1.000
coherence * lag    251.726        1       38    251.726    2.431    0.127        0.127  0.009  1.000


POST HOC TESTS

Contrast    A               B                Paired    Parametric          T     dof  alternative      p-unc    p-corr  p-adjust                         BF10    hedges
----------  --------------  ---------------  --------  ------------  -------  ------  -------------  -------  --------  ----------  -------------------------  --------
condition   coherent_long   coherent_short   True      True          -14.879  38.000  two-sided        0.000     0.000  bonf 

In [9]:
aov = pg.rm_anova(
    data=distance[distance['coherence']!='mainplot'],
    dv='abs_error',
    within=['coherence', 'lag'],
    subject='subject',
    detailed=True
)

pg.print_table(aov)

post_hoc = pg.pairwise_tests(
    data=distance[distance['coherence']!='mainplot'],
    dv='abs_error',
    within='condition',
    subject='subject',
    padjust='bonf'
)

pg.print_table(post_hoc)


ANOVA SUMMARY

Source                 SS    ddof1    ddof2        MS       F    p-unc    p-GG-corr    ng2    eps
---------------  --------  -------  -------  --------  ------  -------  -----------  -----  -----
coherence          39.061        1       38    39.061   0.472    0.496        0.496  0.002  1.000
lag              9801.091        1       38  9801.091  44.714    0.000        0.000  0.304  1.000
coherence * lag   180.040        1       38   180.040   3.110    0.086        0.086  0.008  1.000


POST HOC TESTS

Contrast    A               B                Paired    Parametric         T     dof  alternative      p-unc    p-corr  p-adjust           BF10    hedges
----------  --------------  ---------------  --------  ------------  ------  ------  -------------  -------  --------  ----------  -----------  --------
condition   coherent_long   coherent_short   True      True          -4.802  38.000  two-sided        0.000     0.000  bonf            882.168    -1.063
condition   coher

In [10]:
ttest = pg.ttest(
    x=distance[distance['lag']=='short']['abs_error'],
    y=distance[distance['lag']=='long']['abs_error'],
    paired=True
)

pg.print_table(ttest)

print()

    T    dof  alternative      p-val  CI95%            cohen-d            BF10    power
-----  -----  -------------  -------  -------------  ---------  --------------  -------
8.395     77  two-sided        0.000  [12.09 19.61]      1.307  4696000000.000    1.000




In [11]:
ttest = pg.ttest(
    x=distance[distance['coherence']=='coherent']['abs_error'],
    y=distance[distance['coherence']=='unrelated']['abs_error'],
    paired=True
)

pg.print_table(ttest)

print()

     T    dof  alternative      p-val  CI95%          cohen-d    BF10    power
------  -----  -------------  -------  -----------  ---------  ------  -------
-0.738     77  two-sided        0.463  [-3.7  1.7]      0.069   0.162    0.092




In [13]:
ttest = pg.ttest(
    x=distance[distance['lag']=='short']['error'],
    y=distance[distance['lag']=='long']['error'],
    paired=True
)

pg.print_table(ttest)

print()

     T    dof  alternative      p-val  CI95%            cohen-d                                      BF10    power
------  -----  -------------  -------  -------------  ---------  ----------------------------------------  -------
25.007     77  two-sided        0.000  [36.45 42.76]      2.934  211199999999999984816139969721532416.000    1.000




In [14]:
ttest = pg.ttest(
    x=distance[distance['coherence']=='coherent']['error'],
    y=distance[distance['coherence']=='unrelated']['error'],
    paired=True
)

pg.print_table(ttest)

print()

     T    dof  alternative      p-val  CI95%            cohen-d    BF10    power
------  -----  -------------  -------  -------------  ---------  ------  -------
-1.163     77  two-sided        0.249  [-5.52  1.45]      0.085   0.238    0.114


